In [ ]:
import sys

sys.path.append('..')

from db import *
from analysis import *
from blackbox.helpers import *

# DALL-E-2 Timing Side-Channel

In [ ]:
median_response_time = func.percentile_cont(0.5).within_group(
    ImageCreationRequest.response_time.asc()
).label('median_response_time')

data = (
    session.query(
        Prompts.id.label("Prompt ID"),
        median_response_time.label("Median Response Time"),
        Prompts.dataset.label("Dataset"),
        sql_encode(Prompts.prompt_hash, 'hex').label('Prompt Hash'),
    ).join(
        Prompts, ImageCreationRequest.prompt_id == Prompts.id
    ).filter(
        or_(
            Prompts.dataset == 'blocklist',
            Prompts.dataset == 'blocklist_mutate',
        )
    ).group_by(
        ImageCreationRequest.prompt_id, Prompts.dataset, Prompts.id,
        sql_encode(Prompts.prompt_hash, 'hex').label('prompt_hash'),
    )
)

df = query_to_df(data)

blocklist = df[df['Dataset'] == 'blocklist']
blocklist_mutate = df[df['Dataset'] == 'blocklist_mutate']

df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))

plt_range = (0, df['Median Response Time'].max() *1.25)

ax.hist(blocklist['Median Response Time'], range=plt_range, color=palette[0], label="Original Word", bins=30)
ax.hist(blocklist_mutate['Median Response Time'], range=plt_range, color=palette[1], label="Mutated Words", bins=30)

ax.legend()

ax.set_ylabel('Frequency')
ax.set_xlabel('Median Response Time (s)')

plt.tight_layout()
plt.show()


In [ ]:
# Perform a one-tailed t-test (e.g., testing if blocklist_mutate has higher response times than blocklist)
b1 = blocklist["Median Response Time"]
b2 = blocklist_mutate["Median Response Time"]

t_stat, p_value = stats.ttest_ind(b1, b2, equal_var=False, alternative='two-sided')

results = pd.DataFrame({
    'Metric': ['t-statistic', 'p-value', 'Statistically Significant (p < 0.05)'],
    'Value': [round(t_stat, 4), round(p_value, 4), p_value < 0.05]
})

# Results

**Description:** This experiment demonstrates the effectiveness of our DALL·E 2 blocklist probing attack, which allows an attacker to detect independent filter systems by their respective response times (§5.1). 

**Dataset:** This experiment uses 5 pairs of blocklisted and mutated words, repeated 5 times each, for a total of 50 requests to DALL·E 2.

**Success Criteria:** The experiment demonstrates the statistical significance between rejection timings.

In [ ]:
display(results)